In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
import warnings
warnings.filterwarnings("ignore")
import textwrap

In [ ]:
# use auto mode, automatically select precision based on the device.
#model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-1_8B-Chat", device_map="auto", trust_remote_code=True, fp16=True).eval()

In [3]:
# Note: The default behavior now has injection attack prevention off.
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-1_8B-Chat", trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen-1_8B-Chat-Int4", #"Qwen/Qwen-7B-Chat-Int4",
    device_map="auto",
    trust_remote_code=True
).eval()


Try importing flash-attention for faster inference...


In [4]:
model_size = model.get_memory_footprint()
model_size = model_size / (1024*1024*1024)
# Print model size
print(f"Model size: {model_size:.2f} GB")

Model size: 1.75 GB


In [5]:
response, history = model.chat(tokenizer, "Hi", history=None)
print(textwrap.fill(response, width=80))

Hello! How can I help you today? Is there something you would like to talk about
or ask me a question? I'm here to assist with any information or guidance you
may need. Just let me know and I'll do my best to help.


In [6]:
history

[('Hi',
  "Hello! How can I help you today? Is there something you would like to talk about or ask me a question? I'm here to assist with any information or guidance you may need. Just let me know and I'll do my best to help.")]

In [7]:
response, history = model.chat(tokenizer, "How is the weather today?", history=history)
print(textwrap.fill(response, width=80))

I'm sorry, but as an AI language model, I don't have access to real-time weather
information. However, you can easily check the current weather conditions in
your area by searching for it online or using a weather app on your phone or
computer. You can also find weather forecasts and reports by visiting websites
such as Weather.com or AccuWeather.


In [8]:
history

[('Hi',
  "Hello! How can I help you today? Is there something you would like to talk about or ask me a question? I'm here to assist with any information or guidance you may need. Just let me know and I'll do my best to help."),
 ('How is the weather today?',
  "I'm sorry, but as an AI language model, I don't have access to real-time weather information. However, you can easily check the current weather conditions in your area by searching for it online or using a weather app on your phone or computer. You can also find weather forecasts and reports by visiting websites such as Weather.com or AccuWeather.")]

In [9]:
response, history = model.chat(tokenizer, "What is the capital of Pakistan?", history=None)
print(textwrap.fill(response, width=80))

The capital of Pakistan is Islamabad. It is located on the northern coast of
Pakistan, near the Arabian Sea.


In [10]:
history

[('What is the capital of Pakistan?',
  'The capital of Pakistan is Islamabad. It is located on the northern coast of Pakistan, near the Arabian Sea.')]

In [11]:
response, history = model.chat(tokenizer, "Can you tell us more about the capital city?", history=None)

In [12]:
print(textwrap.fill(response, width=80))
history

I'm sorry, but I'm not sure which capital city you're referring to. Can you
please provide me with more information or context so that I can assist you
better?


[('Can you tell us more about the capital city?',
  "I'm sorry, but I'm not sure which capital city you're referring to. Can you please provide me with more information or context so that I can assist you better?")]

Multiple Tasks

In [14]:
text = f"""Dear Amazon, last week I ordered an Optimus Prime action figure from your online store in Germany. \
Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action \ 
figure of Megatron instead! As a lifelong enemy of the Decepticons, I hope you can understand my dilemma. \ 
To resolve the issue, I demand an exchange of Megatron for the Optimus Prime figure I ordered. \
Enclosed are copies of my records concerning this purchase. I expect to hear from you soon. \
Sincerely, Bumblebee."""

prompt = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple backticks.
2 - Identify the entities in the text delimited by triple backticks
3 - Identify the sentiment of the text
4 - Translate the summary into French.


Separate your answers with line breaks.

Text:
```{text}```
"""
response, history = model.chat(tokenizer, prompt, history=None)
print(textwrap.fill(response, width=80))

```  SUMMARY: Bumblebee received a wrong order and wants an exchange of the
wrong figure.  ENTITIES:  - Amazon  - German online store  - Bumblebee
(sentiment analysis not available in this text)   法语：Un pourpr纨te surprise,
Bumblebee a eu l'erreur de commande et souhaite un changement de trésor.   ```


Sequencing Instructions - Example 1

In [15]:
text1 = f"""
Making a cup of tea is easy! First, you need to get some water boiling. While that's happening, \ 
grab a cup and put a tea bag in it. Once the water is hot enough, just pour it over the tea bag. \ 
Let it sit for a bit so the tea can steep. After a few minutes, take out the tea bag. If you \ 
like, you can add some sugar or milk to taste. And that's it! You've got yourself a delicious \ 
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text1}\"\"\"
"""
response, history = model.chat(tokenizer, prompt, history=None)
print(textwrap.fill(response, width=80))


Step 1 - Get water boiling Step 2 - Grab a cup and put a tea bag in it Step 3 -
Let the water sit for a bit so the tea can steep Step 4 - Take out the tea bag
and add sugar or milk to taste No steps provided.


Sequencing Instructions - Example 2

In [16]:
text2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \ 
walk in the park. The flowers are blooming, and the \ 
trees are swaying gently in the breeze. People \ 
are out and about, enjoying the lovely weather. \ 
Some are having picnics, while others are playing \ 
games or simply relaxing on the grass. It's a \ 
perfect day to spend time outdoors and appreciate the \ 
beauty of nature.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text2}\"\"\"
"""
response, history = model.chat(tokenizer, prompt, history=None)
print(textwrap.fill(response, width=80))

No steps provided.


One-shot learning

In [17]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest valley flows from a modest spring; the \ 
grandest symphony originates from a single note; the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response, history = model.chat(tokenizer, prompt, history=None)
print(textwrap.fill(response, width=80))

Resilience is a quality that helps individuals overcome obstacles and adapt to
change. It involves the ability to bounce back from setbacks and persevere
through difficult times. Resilience also requires a positive attitude,
perseverance, and a willingness to learn from mistakes. Developing resilience
takes practice, but it can greatly benefit an individual's personal growth and
overall well-being.


Summary - Generic

In [18]:
text = f"""
Got this panda plush toy for my daughter's birthday, who loves it and takes it everywhere. It's soft and \ 
super cute, and its face has a friendly look. It's a bit small for what I paid though. I think there \ 
might be other options that are bigger for the same price. It arrived a day earlier than expected, \ 
so I got to play with it myself before I gave it to her.
"""
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words. 

Review: ```{text}```
"""
response, history = model.chat(tokenizer, prompt, history=None)
print(textwrap.fill(response, width=80))

The reviewer bought a panda plush toy for their daughter's birthday. It's soft
and cute, but small for the price. Expected delivery on time, played with it
before giving it to her.


Summary - Shipping Department

In [19]:
text = f"""
Got this panda plush toy for my daughter's birthday, who loves it and takes it everywhere. It's soft and \ 
super cute, and its face has a friendly look. It's a bit small for what I paid though. I think there \ 
might be other options that are bigger for the same price. It arrived a day earlier than expected, \ 
so I got to play with it myself before I gave it to her.
"""
prompt = f"""
Your task is to generate a short summary of a product review from an ecommerce site to give feedback to the \
Shipping deparmtment. 

Summarize the review below, delimited by triple backticks, in at most 30 words, and focusing on any aspects \
that mention shipping and delivery of the product. 

Review: ```{text}```
"""
response, history = model.chat(tokenizer, prompt, history=None)
print(textwrap.fill(response, width=80))

Panda plush toy delivered on time, loved by daughter; came early despite size,
will consider future purchases.


Summary - Pricing Department

In [20]:
text = f"""
Got this panda plush toy for my daughter's birthday, who loves it and takes it everywhere. It's soft and \ 
super cute, and its face has a friendly look. It's a bit small for what I paid though. I think there \ 
might be other options that are bigger for the same price. It arrived a day earlier than expected, \ 
so I got to play with it myself before I gave it to her.
"""
prompt = f"""
Your task is to generate a short summary of a product review from an ecommerce site to give feedback to the \
pricing deparmtment, responsible for determining the price of the product.  

Summarize the review below, delimited by triple backticks, in at most 30 words, and focusing on any aspects \
that are relevant to the price and perceived value. 

Review: ```{text}```
"""
response, history = model.chat(tokenizer, prompt, history=None)
print(textwrap.fill(response, width=80))

Product is cute and soft, face has friendly look, arrives early, may have larger
option. Pricing too high.


Sentiment

In [21]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: '''{text}'''
"""

response, history = model.chat(tokenizer, prompt, history=None)
print(textwrap.fill(response, width=80))

Positive. The reviewer seems satisfied with their purchase, stating that the
panda plush toy is soft, cute, and comes in an earlier-than-expected delivery.


Emotions in message

In [22]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{text}'''
"""
response, history = model.chat(tokenizer, prompt, history=None)
print(textwrap.fill(response, width=80))

- love - takes - soft - cute - arriving early


In [23]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{text}'''
"""

response, history = model.chat(tokenizer, prompt, history=None)
print(textwrap.fill(response, width=80))

No.


In [24]:
response, history = model.chat(tokenizer, "Don't you think that arriving earlier than expected is a good thing and shouldn't make people angry \
                               about it?", history=history)
print(textwrap.fill(response, width=80))


Yes, arriving early can sometimes be seen as an advantage or a positive
experience. However, it is still possible to express anger if the delay caused
inconvenience or frustration. In this case, the reviewer is happy to have
received their item sooner than expected and mentions that they had time to play
with it before giving it to their daughter. Overall, while it may not be an
ideal situation for everyone, it is important to consider the context and
circumstances in which the delay occurred when assessing whether it was positive
or negative.


Proofreading

In [25]:
text = f"""
Got this for my daughter for her birthday cuz she keeps taking mine from my room.  Yes, adults also like pandas too.  She takes \
it everywhere with her, and it's super soft and cute.  One of the ears is a bit lower than the other, and I don't think that was \
designed to be asymmetrical. It's a bit small for what I paid for it though. I think there might be other options that are bigger for \
the same price.  It arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.
"""
prompt = f"proofread and correct this review: ```{text}```"

response, history = model.chat(tokenizer, prompt, history=None)
print(textwrap.fill(response, width=80))

Here is a corrected version of the review: I got this item for my daughter as a
birthday gift because she kept stealing my熊猫 from my room. Yes, adults also like
pandas. She takes it wherever she goes and it's very soft and cute. One ear is
slightly lower than the other, but I don't think it was designed to be
asymmetrical. It's a bit smaller than what I paid for it, but I believe there
may be other options available at the same price. The delivery was on time, so I
was able to play with it myself before giving it to my daughter.


In [26]:
from redlines import Redlines
from IPython.display import display, Markdown, Latex, HTML, JSON

In [27]:
zephyr_response = "I purchased this for my daughter as a birthday gift because she keeps stealing mine from my room. Yes, adults can also enjoy plush pandas. She carries it with her everywhere, and it's incredibly soft and adorable. However, one of the ears appears to be lower than the other, and I don't believe this was intentionally designed to be asymmetrical. Although it's a bit smaller than I anticipated for the price, there might be other options of similar quality that are larger. It arrived a day earlier than expected, allowing me to play with it before gifting it to my daughter."

In [28]:
diff = Redlines(text,zephyr_response)
display(Markdown(diff.output_markdown))

<span style='color:red;font-weight:700;text-decoration:line-through;'>Got </span><span style='color:green;font-weight:700;'>I purchased </span>this for my daughter <span style='color:red;font-weight:700;text-decoration:line-through;'>for her </span><span style='color:green;font-weight:700;'>as a </span>birthday <span style='color:red;font-weight:700;text-decoration:line-through;'>cuz </span><span style='color:green;font-weight:700;'>gift because </span>she keeps <span style='color:red;font-weight:700;text-decoration:line-through;'>taking </span><span style='color:green;font-weight:700;'>stealing </span>mine from my <span style='color:red;font-weight:700;text-decoration:line-through;'>room.  </span><span style='color:green;font-weight:700;'>room. </span>Yes, adults <span style='color:green;font-weight:700;'>can </span>also <span style='color:red;font-weight:700;text-decoration:line-through;'>like pandas too.  </span><span style='color:green;font-weight:700;'>enjoy plush pandas. </span>She <span style='color:red;font-weight:700;text-decoration:line-through;'>takes </span><span style='color:green;font-weight:700;'>carries </span>it <span style='color:red;font-weight:700;text-decoration:line-through;'>everywhere </span>with <span style='color:red;font-weight:700;text-decoration:line-through;'>her, </span><span style='color:green;font-weight:700;'>her everywhere, </span>and it's <span style='color:red;font-weight:700;text-decoration:line-through;'>super </span><span style='color:green;font-weight:700;'>incredibly </span>soft and <span style='color:red;font-weight:700;text-decoration:line-through;'>cute.  One </span><span style='color:green;font-weight:700;'>adorable. However, one </span>of the ears <span style='color:red;font-weight:700;text-decoration:line-through;'>is a bit </span><span style='color:green;font-weight:700;'>appears to be </span>lower than the other, and I don't <span style='color:red;font-weight:700;text-decoration:line-through;'>think that </span><span style='color:green;font-weight:700;'>believe this </span>was <span style='color:green;font-weight:700;'>intentionally </span>designed to be asymmetrical. <span style='color:red;font-weight:700;text-decoration:line-through;'>It's </span><span style='color:green;font-weight:700;'>Although it's </span>a bit <span style='color:red;font-weight:700;text-decoration:line-through;'>small </span><span style='color:green;font-weight:700;'>smaller than I anticipated </span>for <span style='color:red;font-weight:700;text-decoration:line-through;'>what I paid for it though. I think </span><span style='color:green;font-weight:700;'>the price, </span>there might be other options <span style='color:green;font-weight:700;'>of similar quality </span>that are <span style='color:red;font-weight:700;text-decoration:line-through;'>bigger for the same price.  </span><span style='color:green;font-weight:700;'>larger. </span>It arrived a day earlier than expected, <span style='color:red;font-weight:700;text-decoration:line-through;'>so I got </span><span style='color:green;font-weight:700;'>allowing me </span>to play with it <span style='color:red;font-weight:700;text-decoration:line-through;'>myself </span>before <span style='color:red;font-weight:700;text-decoration:line-through;'>I gave </span><span style='color:green;font-weight:700;'>gifting </span>it to my daughter.

In [29]:
diff = Redlines(text,response)
display(Markdown(diff.output_markdown))

<span style='color:red;font-weight:700;text-decoration:line-through;'>Got </span><span style='color:green;font-weight:700;'>Here is a corrected version of the review: </span>

<span style='color:green;font-weight:700;'>I got </span>this <span style='color:green;font-weight:700;'>item </span>for my daughter <span style='color:red;font-weight:700;text-decoration:line-through;'>for her </span><span style='color:green;font-weight:700;'>as a </span>birthday <span style='color:red;font-weight:700;text-decoration:line-through;'>cuz </span><span style='color:green;font-weight:700;'>gift because </span>she <span style='color:red;font-weight:700;text-decoration:line-through;'>keeps taking mine </span><span style='color:green;font-weight:700;'>kept stealing my熊猫 </span>from my <span style='color:red;font-weight:700;text-decoration:line-through;'>room.  </span><span style='color:green;font-weight:700;'>room. </span>Yes, adults also like <span style='color:red;font-weight:700;text-decoration:line-through;'>pandas too.  </span><span style='color:green;font-weight:700;'>pandas. </span>She takes it <span style='color:red;font-weight:700;text-decoration:line-through;'>everywhere with her, </span><span style='color:green;font-weight:700;'>wherever she goes </span>and it's <span style='color:red;font-weight:700;text-decoration:line-through;'>super </span><span style='color:green;font-weight:700;'>very </span>soft and <span style='color:red;font-weight:700;text-decoration:line-through;'>cute.  </span><span style='color:green;font-weight:700;'>cute. </span>One <span style='color:red;font-weight:700;text-decoration:line-through;'>of the ears </span><span style='color:green;font-weight:700;'>ear </span>is <span style='color:red;font-weight:700;text-decoration:line-through;'>a bit </span><span style='color:green;font-weight:700;'>slightly </span>lower than the other, <span style='color:red;font-weight:700;text-decoration:line-through;'>and </span><span style='color:green;font-weight:700;'>but </span>I don't think <span style='color:red;font-weight:700;text-decoration:line-through;'>that </span><span style='color:green;font-weight:700;'>it </span>was designed to be asymmetrical. It's a bit <span style='color:red;font-weight:700;text-decoration:line-through;'>small for </span><span style='color:green;font-weight:700;'>smaller than </span>what I paid for <span style='color:red;font-weight:700;text-decoration:line-through;'>it though. </span><span style='color:green;font-weight:700;'>it, but </span>I <span style='color:red;font-weight:700;text-decoration:line-through;'>think </span><span style='color:green;font-weight:700;'>believe </span>there <span style='color:red;font-weight:700;text-decoration:line-through;'>might </span><span style='color:green;font-weight:700;'>may </span>be other options <span style='color:red;font-weight:700;text-decoration:line-through;'>that are bigger for </span><span style='color:green;font-weight:700;'>available at </span>the same <span style='color:red;font-weight:700;text-decoration:line-through;'>price.  It arrived a day earlier than expected, </span><span style='color:green;font-weight:700;'>price. The delivery was on time, </span>so I <span style='color:red;font-weight:700;text-decoration:line-through;'>got </span><span style='color:green;font-weight:700;'>was able </span>to play with it myself before <span style='color:red;font-weight:700;text-decoration:line-through;'>I gave </span><span style='color:green;font-weight:700;'>giving </span>it to my daughter.